In [2]:
! pip install ccxt

In [34]:
import json
import ccxt
import logging

import pandas as pd
import numpy as np

import time



In [12]:
config = json.load(open('../config.json'))

EXCHANGE = ccxt.binance({
    "apiKey": config["BINANCE_API_KEY"],
    "secret": config["BINANCE_SECRET_KEY"]
})



In [17]:
open_orders = EXCHANGE.fetch_open_orders("LRC/BUSD")

In [20]:
status = open_orders[0]["status"]

'open'

In [24]:
open_orders[0]

{'info': {'symbol': 'LRCBUSD',
  'orderId': '201663864',
  'orderListId': '-1',
  'clientOrderId': 'web_7f66b886f7c94cb0841cf35c5ad93feb',
  'price': '1.67770000',
  'origQty': '12.00000000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': '1641477169031',
  'updateTime': '1641477169031',
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'},
 'id': '201663864',
 'clientOrderId': 'web_7f66b886f7c94cb0841cf35c5ad93feb',
 'timestamp': 1641477169031,
 'datetime': '2022-01-06T13:52:49.031Z',
 'lastTradeTimestamp': None,
 'symbol': 'LRC/BUSD',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 1.6777,
 'stopPrice': None,
 'amount': 12.0,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 12.0,
 'status': 'open',
 'fee': None,
 'trades': [],
 'fees': []}

In [21]:
def select_pairs(pairs, hours):
    """ Sort trading pairs by performance and return dataframe"""
    columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    
    pairs_perc = []

    for i, pair in enumerate(pairs):
        try:
            data = EXCHANGE.fetch_ohlcv(pair, "1h")
            df_temp = pd.DataFrame.from_records(data, columns=columns)
            df_temp.head(12)
            percentage = (df_temp["Close"].iloc[-1] / df_temp["Close"].iloc[-1-hours]) - 1
            pairs_perc.append([pair, percentage])
        except Exception as e:
            logging.warning(f"Could not fetch {pair}")
            logging.warning(f"{e}")
            

    df_pairs_perc = pd.DataFrame(pairs_perc, columns=["pair", "percentage"])
    df_pairs_perc = df_pairs_perc.sort_values(by=['percentage'], ascending=False)
    
    return df_pairs_perc

def filter_pairs(pairs):
    ''' Filter pairs and return only top 200 pairs'''
    # Add filtering for top XXX coins ?
    
    pairs_data = EXCHANGE.fetch_tickers()
    filtered_pairs = []
    
    for pair in pairs: 
        # De if statement kan worden uitgebreid met logica om te zien of het volume substantieel is.
        if pairs_data[pair]["baseVolume"]>0 or pairs_data[pair]["quoteVolume"]>0:
            filtered_pairs.append(pair)
    
    return filtered_pairs

def split_cash(cash, pairs):
    
    tickers = EXCHANGE.fetch_tickers()
    coin_allocation = {}
    isstable = []
    
    for pair in pairs:

        if pair not in isstable:
            ask_price = tickers[pair]["ask"]
            allocation = (1 / len(pairs)) # Allocation buckets maken zodat meerdere trading bots kunnen opereren op hetzelfde account
            available_cash = ( cash * 0.975 ) # Keep some cash in reserve to accomodate for price fluctuations
            coin_amount =  (allocation * available_cash) / ask_price 
            coin_allocation[pair] = [round(coin_amount, 8), ask_price]
        else:
            pass
    
    return coin_allocation

def log_warning(coin, value, bid_price):

    return None



In [36]:
def check_order_status(coin, amount, limit):
    # Kijken hoe we de relevante orders kunnen bijhouden
    # Gebruik maken van parse_dust_trade voor de dust trades
    
    price = amount * limit
    
    if price > 10:
        time.sleep(15)
        open_orders = EXCHANGE.fetch_open_orders(coin)
        if len(open_orders) > 0:
            status = open_orders[0]["status"]
            id = open_orders[0]["id"]

            if status == "open":
                print(f"Order {coin} of {amount} at {limit} not filled!")
                
                
                
                # EXCHANGE.cancel_order(id=id,symbol=coin)
                # EXCHANGE.create_limit_sell_order(coin, amount, limit*0.999)
                print(f"Order cancelled. New order placed at {limit*0.999}")
            
    
    return 
    


def check_market_down():
    return None

In [38]:
def place_market_order(pairs):
    """ Place buy and sell ordered based on trend"""
    
    unfiltered_pairs = pairs["pair"].values
    market_down = pairs["percentage"].values[:100].mean() < 0 # Aanpassen
    
    filtered_pairs = filter_pairs(unfiltered_pairs)
    top_10 = filtered_pairs[:10]
    balance = EXCHANGE.fetch_balance()
    df_balance = pd.DataFrame(balance["free"].items(), columns=["token", "balance"]) # check toevoegen om te zien of het meer dan $10 is 
    portfolio = df_balance[df_balance["balance"]>0]
        
    tickers = EXCHANGE.fetch_tickers()
    
    # Sell
    for token, value in dict(portfolio.values).items():
        if token != "BUSD": 
            coin = f"{token}/BUSD"
            bid_price = tickers[coin]["bid"]
            try:
                print(f"Sell: {coin}, Amount: {value}, Bid: {bid_price}")
                # EXCHANGE.create_limit_sell_order(coin, value, bid_price)
                # EXCHANGE.create_market_sell_order(coin, value)
                check_order_status(coin, value, bid_price)
            except Exception as e:
                logging.warning(f"Could not sell Coin: {coin}, Amount: {value}, Bid:{bid_price}")
                logging.warning(e)
                
    cash = EXCHANGE.fetch_balance()["BUSD"]["free"]
    coin_allocation = split_cash(cash, top_10)
    
    # Buy
    for coin, value in coin_allocation.items():
        if market_down == False: # Check if overall market is down. If True, we will not reenter the market until next period
            try:
                print(f"Buy: {coin}, Amount: {value[0]}, Ask: {value[1]}")
                # EXCHANGE.create_limit_buy_order(coin, value[0], value[1])
                # EXCHANGE.create_market_buy_order(coin, value[0])
            except Exception as e:
                logging.warning(f"Could not buy Coin: {coin}, Amount: {value[0]}, Ask:{value[1]}")
                logging.warning(e)


    
    return None

In [39]:
hours = 8 
pairs = [key for key in EXCHANGE.fetch_tickers().keys() if "/BUSD" in key]

df_pairs_sorted = select_pairs(pairs, hours)
place_market_order(df_pairs_sorted)

Sell: BTC/BUSD, Amount: 3.956e-05, Bid: 42818.27
Sell: LTC/BUSD, Amount: 0.00034013, Bid: 134.8
Sell: LRC/BUSD, Amount: 1.998, Bid: 1.7159
Sell: IOTA/BUSD, Amount: 0.964, Bid: 1.1864
Sell: LINK/BUSD, Amount: 0.00582, Bid: 23.93
Sell: POWR/BUSD, Amount: 0.774, Bid: 0.5266
Sell: REQ/BUSD, Amount: 0.875, Bid: 0.3066
Sell: XMR/BUSD, Amount: 0.00060358, Bid: 197.4
Sell: ENJ/BUSD, Amount: 16.0839, Bid: 2.52
Sell: MANA/BUSD, Amount: 11.988, Bid: 3.2864
Sell: XLM/BUSD, Amount: 0.3472517, Bid: 0.2657
Sell: WAVES/BUSD, Amount: 0.00687, Bid: 13.97
Sell: ELF/BUSD, Amount: 0.0703, Bid: 0.4168
Sell: SYS/BUSD, Amount: 0.873, Bid: 0.9837
Sell: XEM/BUSD, Amount: 0.632, Bid: 0.1214
Sell: IQ/BUSD, Amount: 0.263, Bid: 0.01129
Sell: POLY/BUSD, Amount: 0.0223, Bid: 0.4776
Sell: RVN/BUSD, Amount: 0.04554005, Bid: 0.11735
Sell: FET/BUSD, Amount: 0.752, Bid: 0.5407
Sell: CELR/BUSD, Amount: 0.429, Bid: 0.07429
Sell: ATOM/BUSD, Amount: 0.00104, Bid: 37.26
Sell: ONE/BUSD, Amount: 0.0514, Bid: 0.28078
Sell: FTM/BU